In [ ]:
%matplotlib inline
# %matplotlib

In [ ]:
# Loading libaries

import datetime
import os
import pickle
import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors 
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('png')

plt.rcParams['savefig.dpi'] = 75
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = (21, 30)  # A4
plt.rcParams['lines.linewidth'] = 1.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['text.usetex'] = False
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['xtick.major.pad']=12
matplotlib.rcParams['ytick.major.pad']=12
#matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

import numpy as np
import scipy as scipy
import scipy.stats as stats
import scipy.linalg as linalg
import xarray as xr
from pprint import pprint
import sys
import os
import copy
import string
import glob
#import xarray as xr 

import warnings
#warnings.filterwarnings('ignore')

In [ ]:
base_path = os.path.abspath('L63_FTLEs.ipynb')
#base_dir = '/g/data/v14/Projects/L63_dimension'

In [ ]:
now = datetime.datetime.now()
print('date : {0} / {1} / {2}'.format(now.day, now.month, now.year))
cwd = os.getcwd()
print('working directory : {0}'.format(cwd))
print('data directory : {0}'.format(cwd))

In [ ]:
cmaps = [('Perceptually Uniform Sequential', [
            'viridis', 'plasma', 'inferno', 'magma', 'cividis']),
         ('Sequential', [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']),
         ('Sequential (2)', [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper']),
         ('Diverging', [
            'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic',
            'berlin', 'managua', 'vanimo']),
         ('Cyclic', ['twilight', 'twilight_shifted', 'hsv']),
         ('Qualitative', [
            'Pastel1', 'Pastel2', 'Paired', 'Accent',
            'Dark2', 'Set1', 'Set2', 'Set3',
            'tab10', 'tab20', 'tab20b', 'tab20c']),
         ('Miscellaneous', [
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
            'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
            'gist_ncar'])]

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(cmap_category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=nrows, figsize=(6.4, figh))
    fig.subplots_adjust(top=1-.35/figh, bottom=.15/figh, left=0.2, right=0.99)

    axs[0].set_title(f"{cmap_category} colormaps", fontsize=14)

    for ax, cmap_name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=cmap_name)
        ax.text(-.01, .5, cmap_name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()


#for cmap_category, cmap_list in cmaps:
 #   plot_color_gradients(cmap_category, cmap_list)
#plt.show()

In [ ]:
from DA_module import *
from ftles import *
from CLVs import *

# Control Run

In [ ]:
# Model properties
dt                = 0.005

parameters_spinup = 50     # skip this first number of analyses in determining statistics
forecast_window   = 25       # window between each DA cycle

dim  = 3

# control run
num_steps = 100000
np.random.seed(seed=1)
IC     = np.ones(dim)
IC_run = Lorenz63(num_time_steps=num_steps, dt=dt, IC=IC)
IC_run.run()

In [ ]:
fig, a1 = plt.subplots(1,1,figsize=[20,10])
a1.plot(IC_run.time[:50000],IC_run.x[:,:50000].T)
a1.grid()
a1.set_xlabel('time')
plt.show()

In [ ]:
fig = plt.figure(figsize=[10,10],layout='constrained',dpi=300)
a1 = fig.add_subplot(1,1,1,projection='3d')
a1.plot(IC_run.x[0,5000:],IC_run.x[1,5000:],IC_run.x[2,5000:],'.',ms=0.5)
a1.grid()
plt.savefig('Results/L63/2D axes view.png',format="png",bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[20,20],layout='constrained',dpi=300)
dim2 = 3
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1)
        a1.plot(IC_run.x[i,:],IC_run.x[j,:],'.',ms=0.5)
        a1.grid()
        a1.set_title('{}-{} axes'.format(i+1,j+1))

plt.savefig('Results/L63/2D axes view.png',format="png",bbox_inches='tight')
plt.show()  

# Dynamical Properties

In [ ]:
start_ind = 4000
start_time = dt*start_ind

start = time.time()
cocycle = IC_run.calculate_matrix_cocycle(IC_run.x,tlims=[start_time,dt*num_steps])
end = time.time()

comp_time = (end-start)

print('cocycle calculated in ' + str(np.round(comp_time,4)) + ' sec')

## FTLEs

In [ ]:
CLVS_DIR = os.path.join(cwd, 'CLVs')
    
if not os.path.exists(CLVS_DIR):
    os.makedirs(CLVS_DIR)

Rdiag_filename = '.'.join(['L63Rdiag','nc'])
Rdiag_file = os.path.join(CLVS_DIR, Rdiag_filename)

#if os.path.exists(Rdiag_file):
#    Rdiag = xr.open_dataset(Rdiag_file).Rdiag
#else:
start = time.time()
Rdiag = calculate_FTLEs(dim, cocycle, np.arange(0,cocycle.shape[2]))[1]
end = time.time()

comp_time = (end-start)/60

print('R matrices calculated in ' + str(np.round(comp_time,4)) + ' min')
    
    ## convert to xarray
Rdiag = xr.DataArray(Rdiag)
    
    # save to netCDF
Rdiag.to_dataset(name='L63Rdiag').to_netcdf(Rdiag_file)

In [ ]:
window = 25

FTLEs = np.zeros((dim,num_steps-start_ind-window))

start = time.time()
for ti in np.arange(0,num_steps-window-start_ind):
    FTLEs[:,ti] = np.sum(np.log(Rdiag[:,ti:ti+window]),axis=1)/(window*dt)
end = time.time()
comp_time = (end-start)

print('calculated in ' + str(np.round(comp_time,4)) + ' sec')

In [ ]:
mycmap = matplotlib.colors.ListedColormap([
 [0., 0., 1., 1.      ],
 [0.25, 0.25, 1., 1.      ],
 [0.5, 0.5, 1., 1.      ], 
 [0.75, 0.75, 1., 1.      ],
 [1., 0.75, 0.75, 1.      ],
 [1., 0.5, 0.5, 1.      ],
 [1., 0.25, 0.25, 1.      ],
 [1., 0., 0., 1.      ]])

fig = plt.figure(figsize=[24,10],layout='constrained',dpi=300)
a1 = fig.add_subplot(1,3,1,projection='3d')
scat = a1.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[0,:],s=2,cmap='RdBu_r',vmax=8,vmin=-8)
a1.grid()
#a1.set_xlabel('x')
#a1.set_ylabel('y')
#a1.set_zlabel('z')
a1.set_title('FTLE 1',fontsize=24)
cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('FTLE 1')

a2 = fig.add_subplot(1,3,2,projection='3d')
scat = a2.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[1,:],s=2,cmap='RdBu_r',vmax=5,vmin=-5)
a2.grid()
#a2.set_xlabel('x')
#a2.set_ylabel('y')
#a2.set_zlabel('z')
cbar = plt.colorbar(scat,shrink=0.7)
a2.set_title('FTLE 2',fontsize=24)
#cbar.set_label('FTLE 2')

a3 = fig.add_subplot(1,3,3,projection='3d')
scat = a3.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[2,:],s=2,cmap='RdBu_r',vmax=20,vmin=-20)
a3.grid()
#a3.set_xlabel('x')
#a3.set_ylabel('y')
#a3.set_zlabel('z')
a3.set_title('FTLE 3',fontsize=24)
cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('FTLE 3')


plt.savefig('Results/L63/FTBLEs_wind{}.png'.format(window),format="png",bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,24])
a1 = fig.add_subplot(3,1,1)
scat = a1.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[0,:20000],s=2,cmap='RdBu_r',vmax=10,vmin=-10)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 1')

a2 = fig.add_subplot(3,1,2)
scat = a2.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[1,:20000],s=2,cmap='RdBu_r',vmax=4,vmin=-4)
a2.grid()
a2.set_xlabel('x')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 2')

a3 = fig.add_subplot(3,1,3)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[2,:20000],s=2,cmap='RdBu_r',vmax=4,vmin=-4)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 3')

plt.show()


In [ ]:
np.mean(FTLEs,axis=1)

## CLVs

In [ ]:
M = 25
orth = 1
num_CLVs = cocycle.shape[2]-2*M

CLVs_filename = '.'.join(['L63CLVs', 'M{:d}'.format(M),'orth{:d}'.format(orth),'nc'])
CLVs_file = os.path.join(CLVS_DIR, CLVs_filename)

#if os.path.exists(CLVs_file):
#    CLVs = xr.open_dataset(CLVs_file).CLVs
#else:
    # Calculate CLVs
Nk = np.arange(0,M+1,orth) 

CLVs = np.zeros((dim,dim,num_CLVs),dtype=np.float64)

start = time.time()
for ti in np.arange(0,num_CLVs):
    CLVs[:,:,ti] = calculate_CLV_numerically(dim, cocycle[:,:,ti:ti+2*M], Nk, M)
end = time.time()

comp_time = (end-start)/60

print('calculated in ' + str(np.round(comp_time,4)) + ' min')
    
    ## convert to xarray
CLVs = xr.DataArray(CLVs, coords=[np.arange(1,CLVs.shape[0]+1),np.arange(1,CLVs.shape[1]+1), np.arange(M,M+num_CLVs)],
                               dims=['coordinate','CLV', 'timestep'])
    
    ## save to netCDF file
CLVs.to_dataset(name='L63CLVs').to_netcdf(CLVs_file)

In [ ]:
# Calculate alignment
align = np.array(np.zeros((dim,dim,num_CLVs),dtype=np.float64))

start = time.time()
for clvi in np.arange(0,dim):
    for clvj in np.arange(0,dim):
        for tt in np.arange(0,num_CLVs):
            align[clvi,clvj,tt] = abs(np.dot(CLVs.values[:,clvi,tt],CLVs.values[:,clvj,tt]))
            
end = time.time()
comp_time = (end-start)

print('alignment calculated in ' + str(np.round(comp_time,4)) + ' sec')

In [ ]:
mycmap2 = matplotlib.colors.ListedColormap([
    [1., 0.8, 0.8, 1.      ],
    [1., 0.6, 0.6, 1.     ],
    [1., 0.4, 0.4, 1.      ], 
    [1., 0.2, 0.2, 1.      ],
    [1., 0., 0., 1.]])        
    
fig = plt.figure(figsize=[24, 10],layout='constrained',dpi=300)
a1 = fig.add_subplot(1,3,1,projection='3d')
a2 = fig.add_subplot(1,3,2,projection='3d')
a3 = fig.add_subplot(1,3,3,projection='3d')
#cmap = matplotlib.cm.get_cmap('seismic',10)
#cmap_opaque = np.array(np.zeros((10,4),dtype=np.float))
#for c in np.arange(0,10):
#    cmap_opaque[c,:] = np.array(cmap(c))
#    if c < 7:
#        cmap_opaque[c,3] = 0.5
#cmap = matplotlib.colors.ListedColormap(cmap_opaque)
#cmap.set_bad('w',1.)
    
#cax = a1.matshow(align.T,cmap=cmap,vmin=0,vmax=1)

scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=align[0,1,:],s=2,cmap='RdBu_r',vmax=1,vmin=0)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
a1.set_zlabel('z')
a1.set_title('CLV 1&2',fontsize=24)

scat = a2.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=align[0,2,:],s=2,cmap='RdBu_r',vmax=1,vmin=0)
a2.grid()
a2.set_xlabel('x')
a2.set_ylabel('y')
a2.set_zlabel('z')
a2.set_title('CLV 1&3',fontsize=24)

scat = a3.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=align[1,2,:],s=2,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
a3.set_zlabel('z')
a3.set_title('CLV 2&3',fontsize=24)


cbar = plt.colorbar(scat,shrink=0.7)
cbar.set_label('alignment')

#cbar_ax = fig.add_axes([0.93, 0.13, 0.01, 0.75])
#fig.colorbar(scat, cax=cbar_ax)

plt.savefig('Results/L63/alignment_FTBLEs_wind{}.png'.format(window),format="png",bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,18])
a1 = fig.add_subplot(3,1,1)
scat = a1.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,1,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&2')

a2 = fig.add_subplot(3,1,2)
scat = a2.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,2,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a2.grid()
a2.set_xlabel('x')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&3')

a3 = fig.add_subplot(3,1,3)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[1,2,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 2&3')
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 3
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1)
        scat = a1.scatter(IC_run.x[i,:95600],IC_run.x[j,:95600],
        c=align[i,j,:95600],s=2,cmap='RdBu_r',vmax=1,vmin=0)
        a1.grid()
        a1.set_title('{}-{} axes'.format(i+1,j+1),fontsize=24)
        
cbar = plt.colorbar(scat,shrink=0.7)
cbar.set_label('alignment',fontsize=24)
plt.savefig('Results/L63/alignment 2D axes view M{}_orth{}_wind{}.png'.format(M,orth,window),format="png",bbox_inches='tight')
plt.show()  

## Growth rates CLVs

In [ ]:
M_GR  = 25
orth_win = 1
Nk = np.arange(orth_win-1,M_GR,orth_win)
GRs = np.empty((CLVs.shape[1],CLVs.shape[2]))

start = time.time()

for i in np.arange(0,CLVs.shape[2]):
    C = np.array(CLVs[:,:,i])
    norm_C = linalg.norm(C,axis=0)
    GR_i = np.zeros((CLVs.shape[1],Nk.shape[0]))
    ni = 0
    for tt in np.arange(0,M_GR):
        C = np.matmul(cocycle[:,:,i+tt+M],C)
        if np.isin(tt,Nk):
            GR_i[:,ni] = (linalg.norm(C,axis=0)-norm_C) / (norm_C * dt * orth_win)
            norm_C = linalg.norm(C,axis=0)
            #C[:,1] = C[:,1] - (np.dot(C[:,0], C[:,1]) / np.dot(C[:,0], C[:,0]))*C[:,0]
            #C[:,2] = C[:,2] - (np.dot(C[:,1], C[:,2]) / np.dot(C[:,1], C[:,1]))*C[:,1]
            ni += 1
    #GRs[:,i] = (linalg.norm(C,axis=0)-norm_C)/norm_C
    #GRs[:,i] = GR_i[:,-1]
    GRs[:,i] = np.mean(GR_i,axis=1)

end = time.time()
elapsed = (end-start)/60
print("Elapsed time: {} min, M = {}".format(round(elapsed,4),M_GR)) 

In [ ]:
fig = plt.figure(figsize=[24,10],layout='constrained',dpi=300)

vlims = [[8,-8],[5,-5],[20,-20]]

for i in np.arange(0,dim):
    a1 = fig.add_subplot(1,3,i+1,projection='3d')
    scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=GRs[i,:],s=2,cmap='RdBu_r',vmax=vlims[i][0],vmin=vlims[i][1])
    a1.grid()
    #a1.set_xlabel('x')
    #a1.set_ylabel('y')
    #a1.set_zlabel('z')
    a1.set_title('GR {}'.format(i+1),fontsize=24)
    cbar = plt.colorbar(scat,shrink=0.7)
    #cbar.set_label('FTLE {}'.format(i+1))
    
plt.savefig('Results/L63/FTCLEs_M{}_orth{}_M_GR{}.png'.format(M,orth,M_GR),format="png",bbox_inches='tight')
plt.show()

In [ ]:
np.mean(GRs,axis=1)

## Kaplan-Yorke Dimension

In [ ]:
#FTLE_type = 'FTBLEs'
FTLE_type = 'FTCLEs'  # 'FTBLEs' or 'FTCLEs'

if FTLE_type == 'FTCLEs':
    KY_FTLEs = GRs
elif FTLE_type == 'FTBLEs':
    KY_FTLEs = FTLEs
else:
    print('error: KY_FTLEs type not defined')  
    

dimKY = np.empty((KY_FTLEs.shape[1]))
t = 0
    
start = time.time()
for tt in np.arange(0,KY_FTLEs.shape[1]):
    Lyaps_t = KY_FTLEs[:,tt]
    Lyaps_ord = np.sort(Lyaps_t)
    Lyaps_ord = Lyaps_ord[::-1]
    for ll in np.arange(1,Lyaps_ord.shape[0]):
        S = np.sum(Lyaps_ord[:ll])
        if S<0:
            i_min = ll
            break
        elif ll == dim:
            i_min = ll
    
    dimKY[t] = i_min + np.sum(Lyaps_ord[:i_min])/abs(Lyaps_ord[i_min])
    t += 1
    
end = time.time()
elapsed = end-start
print("Elapsed time: {} sec".format(round(elapsed,4),M))


In [ ]:
mycmap3 = matplotlib.colors.ListedColormap([
    [0.7, 0.6, 1., 1.      ],
    [0.7, 0.6, 1., 1.      ],
    [0.5, 0.3, 0.8, 1.     ],
    [0.5, 0.3, 0.8, 1.      ], 
    [0.3, 0., 0.6, 1.      ],
    [0.3, 0., 0.6, 1.]]) 

fig = plt.figure(figsize=[10,10],layout='constrained',dpi=300)
a1 = fig.add_subplot(1,1,1,projection='3d')

if FTLE_type == 'FTBLEs':
    scat = a1.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=dimKY,s=2,cmap='RdBu_r',vmax=3,vmin=0)
elif FTLE_type == 'FTCLEs':
    scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=dimKY,s=2,cmap='RdBu_r',vmax=3,vmin=0)
else:
     print('error: KY_FTLEs type not defined')   
a1.grid()
#a1.set_xlabel('x')
#a1.set_ylabel('y')
#a1.set_zlabel('z')
a1.set_title('dimKY')

cbar = plt.colorbar(scat,shrink=0.7)
#cbar.set_label('dimKY')

if FTLE_type == 'FTBLEs':
    print('save command commented') 
    plt.savefig('Results/L63/dimKY_FTBLEs_wind{}.png'.format(window),format="png",bbox_inches='tight')
elif FTLE_type == 'FTCLEs':
    print('save command commented') 
    plt.savefig('Results/L63/dimKY_FTCLEs_M{}_M_GR{}_orth{}.png'.format(M,M_GR,orth_win),format="png",bbox_inches='tight')
else:
     print('error: KY_FTLEs type not defined')  

plt.show()    

In [ ]:
fig = plt.figure(figsize=[24,18])
a1 = fig.add_subplot(3,1,1)
scat = a1.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=0,vmin=3)
a1.grid()
a1.set_xlabel('time')
a1.set_ylabel('x')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a2 = fig.add_subplot(3,1,2)
scat = a2.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[1,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=0,vmin=3)
a2.grid()
a2.set_xlabel('time')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a3 = fig.add_subplot(3,1,3)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[2,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=0,vmin=3)
a3.grid()
a3.set_xlabel('time')
a3.set_ylabel('z')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')
plt.show()

In [ ]:
#25/0.